In [5]:
%matplotlib inline
from netpyne import specs, sim
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [6]:
# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters


## Cell parameters
netParams.cellParams['pyr'] = {
    'secs': {
        'soma': {
            'geom': {
                'diam': 18.8,   
                'L': 18.8, 
                'Ra': 123.0},
            'mechs': {
                'hh': {
                    'gnabar': 0.12, 
                    'gkbar': 0.036, 
                    'gl': 0.0003, 
                    'el': -70}
            },
            'ions':{
                'k': {
                    'e': -85,
                }
                
            }
        }
    }
}                


## Population parameters
netParams.popParams['E'] = {
    'cellType': 'pyr', 
    'numCells': 40}

netParams.popParams['I'] = {
    'cellType': 'pyr', 
    'numCells': 10}


## Synaptic mechanism parameters
netParams.synMechParams['exc'] = {
    'mod': 'Exp2Syn', 
    'tau1': 0.1, 
    'tau2': 5.0, 
    'e': 0}  # excitatory synaptic mechanism

netParams.synMechParams['inh'] = {
    'mod': 'Exp2Syn', 
    'tau1': 0.1, 
    'tau2': 5.0, 
    'e': -70}  # inhibitory synaptic mechanism


# Stimulation parameters
netParams.stimSourceParams['bkg'] = {
    'type': 'NetStim', 
    'rate': 50, 
    'noise': 0.5}
    
netParams.stimTargetParams['bkg->E'] = {
    'source': 'bkg', 
    'conds': {'pop': 'E'}, 
    'weight': 0.01, 
    'delay': 5, 
    'synMech': 'exc'}


## Connectivity rules
netParams.connParams['E->I'] = {    #  S -> I label
    'preConds': {'pop': 'E'},       # conditions of presyn cells
    'postConds': {'pop': 'I'},      # conditions of postsyn cells
    'divergence': 5,               # probability of connection
    'weight': 0.01,                 # synaptic weight
    'delay': 5,                     # transmission delay (ms)
    'synMech': 'exc'}               # synaptic mechanism

netParams.connParams['I->E'] = {    #  I -> S label
    'preConds': {'pop': 'I'},       # conditions of presyn cells
    'postConds': {'pop': 'E'},      # conditions of postsyn cells
    'probability': 0.7,               # probability of connection
    'weight': 0.02,                 # synaptic weight
    'delay': 5,                     # transmission delay (ms)
    'synMech': 'inh'}               # synaptic mechanism


# Simulation options
simConfig = specs.SimConfig()       # object of class SimConfig to store simulation configuration

simConfig.duration = 25*1e3          # Duration of the simulation, in ms
simConfig.dt = 0.01                # Internal integration timestep to use
simConfig.verbose = False           # Show detailed messages
simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
simConfig.recordStep = 0.1        # Step size in ms to save data (eg. V traces, LFP, etc)
simConfig.filename = 'tut_oscillation'  # Set file output name
simConfig.saveJson = False

simConfig.recordLFP = [[50, 50, 50]]
simConfig.recordDipole = True

simConfig.analysis['plotTraces'] = {'include': [1], 'saveFig': False}  # Plot recorded traces for this list of cells
simConfig.analysis['plotRaster'] = {'showFig': False}                  # Plot a raster
simConfig.analysis['plotSpikeHist'] = {'include': ['E', 'I'], 'showFig': False}
simConfig.analysis['plotRateSpectrogram'] = {'include': ['all']}


# Create network and run simulation
# sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)

In [ ]:
qtd_sample = 1
all_eeg = []
targets = []
for param_k in tqdm([-55, -85]):
    #netParams.cellParams['pyr']["secs"]["soma"]["ions"]["k"]["e"] = param_k
    for j in range(qtd_sample):
        netParams.cellParams['pyr']["secs"]["soma"]["ions"]["k"]["e"] = param_k + np.random.randn()*0.1
        netParams.stimSourceParams['bkg']['rate'] = 50+np.random.randn()*0.1
        netParams.stimSourceParams['bkg']['noise'] = 0.5+np.random.randn()*0.01
        sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)
        array, closest_elec_idx = sim.analysis.plotEEG()
#         plt.plot(array[closest_elec_idx,-1000:])
        all_eeg.append(array[closest_elec_idx,2000000::560])
        targets.append(param_k)

  0%|                                                     | 0/2 [00:00<?, ?it/s]


Start time:  2023-01-18 18:49:54.801406

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.00 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.04 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.00 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...


In [ ]:
plt.plot(all_eeg[0])

In [ ]:
plt.plot(all_eeg[-1][:1000])

In [ ]:
plt.plot(all_eeg[0])